# Example to Train, Save and Reuse Tensorflow Model:

In [3]:
import tensorflow as tf
import sklearn.datasets
import numpy as np
import pickle
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
n_nodes_h1 = 100
n_nodes_h2 = 100
n_classes = 2
n_features = 2

data_size = 1600
batch_size = 100
total_batches = int(data_size/batch_size)

hm_epochs = 10

tf_log = 'tf1.log'

X, y_labels = sklearn.datasets.make_moons(data_size, noise= 0.20)
X = X.astype('float32')
y_ = y_labels.astype('float32')
y_ = (np.arange(n_classes) == y_[:, None]).astype(np.float32)

## Our Data

<img src="moon1.png" style="width:60%;height:60%">

In [5]:
g = tf.Graph()
with g.as_default():
    x = tf.placeholder('float32')
    y = tf.placeholder('float32')
    
    layer1 = {'hm_nodes':n_nodes_h1,
              'weights':tf.Variable(tf.random_normal([n_features, n_nodes_h1])),
              'bias':tf.Variable(tf.random_normal([n_nodes_h1]))}
    layer2 = {'hm_nodes':n_nodes_h2,
              'weights':tf.Variable(tf.random_normal([n_nodes_h1, n_nodes_h2])),
              'bias':tf.Variable(tf.random_normal([n_nodes_h2]))}
    output_layer = {'hm_nodes':None,
                    'weights':tf.Variable(tf.random_normal([n_nodes_h2, n_classes])),
                    'bias':tf.Variable(tf.random_normal([n_classes]))}

In [6]:
def our_model(data_x):
    with g.as_default():
        l1 = tf.add(tf.matmul(data_x, layer1['weights']), layer1['bias'])
        l1 = tf.nn.relu(l1)
        
        l2 = tf.add(tf.matmul(l1, layer2['weights']), layer2['bias'])
        l2 = tf.nn.relu(l2)
        
        output = tf.add(tf.matmul(l2, output_layer['weights']), output_layer['bias'])
        return output
        

In [7]:
with g.as_default():
    our_saver = tf.train.Saver()

In [8]:
'''It will start from 1 or where you left it last and go ~hm_epochs~ more times'''
def train_model(x):    
    with g.as_default():       
        
        prediction = our_model(x)
        
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction, y))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
        
        
    with tf.Session(graph=g) as sess:
        sess.run(tf.initialize_all_variables())
        hm_epochs = 2
        try:
            epoch = int(open(tf_log, 'r').read().split('\n')[-2]) + 1            
            print('Starting Epoch: ', epoch)
        except Exception as e:
            epoch = 1
            
        
        hm_epochs += epoch
        while epoch <= hm_epochs:
            
            if epoch != 1:
                print('Found session..')
                print('Loading saved session...')
                our_saver.restore(sess, 'model1.ckpt')
            
            epoch_loss = 1
            batches_run = 0
            
            while batches_run < total_batches:
                batch_x = X[batch_size * batches_run: batch_size * (batches_run+1), :]                
                batch_y = y_[batch_size * batches_run: batch_size * (batches_run+1)]
                
                feed_dict = {x:batch_x,
                             y:batch_y}
                
                _, c = sess.run([optimizer, cost], feed_dict)
                epoch_loss += c
                batches_run += 1
#               print('Batch run:',batches_run,'/',total_batches,'| Epoch:',epoch,'| Batch Loss:',c)
            
            
            our_saver.save(sess, 'model1.ckpt')
            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)
            print('Saving session....')
            print('---------------------------------------------------------------')
            
            with open(tf_log, 'a') as f:
                f.write(str(epoch)+'\n')
            epoch += 1
            

In [9]:
with g.as_default():
    train_model(x)

Starting Epoch:  192
Found session..
Loading saved session...
Epoch 192 completed out of 194 loss: 3.24465046078
Saving session....
---------------------------------------------------------------
Found session..
Loading saved session...
Epoch 193 completed out of 194 loss: 2.93509960547
Saving session....
---------------------------------------------------------------
Found session..
Loading saved session...
Epoch 194 completed out of 194 loss: 2.77513140347
Saving session....
---------------------------------------------------------------


# Result of Trained Model

<img src="moon2.png" style="width:80%;height:60%">


In [10]:
with g.as_default():
    train_model(x)

Starting Epoch:  195
Found session..
Loading saved session...
Epoch 195 completed out of 197 loss: 3.2467046082
Saving session....
---------------------------------------------------------------
Found session..
Loading saved session...
Epoch 196 completed out of 197 loss: 2.82921672054
Saving session....
---------------------------------------------------------------
Found session..
Loading saved session...
Epoch 197 completed out of 197 loss: 2.72800714709
Saving session....
---------------------------------------------------------------


In [11]:
with g.as_default():
    train_model(x)

Starting Epoch:  198
Found session..
Loading saved session...
Epoch 198 completed out of 200 loss: 2.731184843
Saving session....
---------------------------------------------------------------
Found session..
Loading saved session...
Epoch 199 completed out of 200 loss: 2.61800396256
Saving session....
---------------------------------------------------------------
Found session..
Loading saved session...
Epoch 200 completed out of 200 loss: 2.68504626211
Saving session....
---------------------------------------------------------------
